# AS - TME7
2018--2019

---

Binome : Juliette Ortholand, Stieban Fernandez

Modèles de séquence (réccurents)
- Modèle sur les caractères (prédire classe)
- Modèle génératif

Données Page AS@DAC
-> names (classes) :
- english.txt
- french.txt
- italian.txt
- ...

Chaque caractère en 1-hot -> char-embedding

Etat précédent : s0 s1, s2, ... application d'une fonction f

+

Classifieur

On travaille avec des batchs

-> remplir caractère vide par 0 par ex

---

Pour les modèles génératifs : longueur du prénom + 1 caractère pour déterminer la fin de la chaine :

Appliquer un masque : 1 s'il y a un caractère (+ caractère de fin) sinon 0

-> Prédire à chaque fois l'état (/caractère) suivant.

Utilisation : 
- Est-ce que "Pierre" est un prénom anglais ? (Calcul de la vraisemblance)
- Générer des prénoms

---

Parcourir les fichiers du répértoire avec glob('*.txt')

In [1]:
import torch
import torch.nn as nn
from unidecode import unidecode

In [8]:
from pathlib import Path

gen = Path("data/names").glob("*.txt")

L = []
D = {}
for label, file in enumerate(gen):
    
    lang = str(file).split("\\")[-1].split(".")[0]
    D[lang] = label
    
    with open(str(file), "r",encoding = 'utf8') as f:
        names = f.read().split("\n")
    for name in names:
        # Nettoyage
        name = "".join([c for c in unidecode(name.lower()) if c.isalpha()])
        L.append((name, label))
        
D, L

({'Arabic': 0,
  'Chinese': 1,
  'Czech': 2,
  'Dutch': 3,
  'English': 4,
  'French': 5,
  'German': 6,
  'Greek': 7,
  'Irish': 8,
  'Italian': 9,
  'Japanese': 10,
  'Korean': 11,
  'Polish': 12,
  'Portuguese': 13,
  'Russian': 14,
  'Scottish': 15,
  'Spanish': 16,
  'Vietnamese': 17},
 [('khoury', 0),
  ('nahas', 0),
  ('daher', 0),
  ('gerges', 0),
  ('nazari', 0),
  ('maalouf', 0),
  ('gerges', 0),
  ('naifeh', 0),
  ('guirguis', 0),
  ('baba', 0),
  ('sabbagh', 0),
  ('attia', 0),
  ('tahan', 0),
  ('haddad', 0),
  ('aswad', 0),
  ('najjar', 0),
  ('dagher', 0),
  ('maloof', 0),
  ('isa', 0),
  ('asghar', 0),
  ('nader', 0),
  ('gaber', 0),
  ('abboud', 0),
  ('maalouf', 0),
  ('zogby', 0),
  ('srour', 0),
  ('bahar', 0),
  ('mustafa', 0),
  ('hanania', 0),
  ('daher', 0),
  ('tuma', 0),
  ('nahas', 0),
  ('saliba', 0),
  ('shamoon', 0),
  ('handal', 0),
  ('baba', 0),
  ('amari', 0),
  ('bahar', 0),
  ('atiyeh', 0),
  ('said', 0),
  ('khouri', 0),
  ('tahan', 0),
  ('baba', 0

In [40]:


cod_char = lambda x: ord(x)-96

def encode_name(name,n):
    res =[cod_char(x) for x in name]
    for i in range(n-len(res)):
        res.append(0)
    return res

def data_loader(data, batch_size,n):
    emb = nn.Embedding(27,10)
    x = []
    y = []
    for name,label in data:
        encod = torch.tensor([encode_name(name,n)])
        x.append(emb(encod).tolist())
        y.append(label)
    x = torch.tensor(x)
    y = torch.tensor(y)
    return x,y

In [44]:
x,y = data_loader(L, 1,20)

In [47]:
x[0][0][0]

tensor([-1.3652,  0.5527, -2.1435, -0.7498, -2.6038,  1.6770, -1.0571, -0.5050,
         0.2788,  1.0850])

In [101]:
class Rec(nn.Module):
    
    def __init__(self):
        super(Rec, self).__init__()
        self.seq1 = nn.Sequential(
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
            nn.Linear(15, 5),
        )
       
    def forward(self, X):
        init = torch.zeros(len(x),5)
        interm = torch.cat([init,X[:,0,0]],1)
        out = []
        for i in range(len(X[0][0])):
            res = self.seq1[i](interm)
            out.append(res.view(1,len(res),len(res[0])))
            interm = torch.cat([res,X[:,0,i]],1)
        return torch.cat(out)
    
    def score(self,ypred,y):
        score = 0
        if len(y) != len(ypred):
            print('ERREUR')
        for i in range(0,len(y)):
            if y[i] == ypred[i].argmax():
                score += 1
        return(score/len(y))

In [102]:
model = Rec()

In [104]:
out = model.forward(x)

torch.Size([20092, 5])
torch.Size([20092, 10])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
torch.Size([20092, 5])
[tensor([[[-0.7002,  0.8043, -0.3514, -0.6605,  1.3375],
         [-0.1372,  0.0012,  0.4365, -0.5752,  0.1200],
         [ 0.1077,  0.4947, -0.8994, -0.3886,  0.0094],
         ...,
         [ 0.2608, -0.3554,  0.1433, -0.5188,  0.1890],
         [ 0.2608, -0.3554,  0.1433, -0.5188,  0.1890],
         [-0.1793,  1.1050,  0.5620,  0.5352, -0.3682]]],
       grad_fn=<ViewBackward>), tensor([[[ 0.4857,  0.7307,  0.7511,  1.0243, -1.0263],
         [ 0.1898,  0.1489,  0.0661,  0.5682,  0.080

In [105]:
out.shape

torch.Size([20, 20092, 5])

In [71]:
out = out.view(1,len(out))

In [72]:
torch.cat([out,out])

tensor([[0., 0.],
        [0., 0.]])